# 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import f1_score

In [2]:
data = pd.read_csv('creditcard.csv')

In [3]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
Xdata = data.drop('Class', axis=1)
Ydata = data.Class

In [5]:
sum(data.Class)/len(data.Class) # imbalanced data

0.001727485630620034

# LGBM

In [6]:
from lightgbm import LGBMClassifier

In [7]:
lgbm = LGBMClassifier(num_leaves=100, learning_rate=0.001, n_estimators=1000, objective='binary')

num_leaves: 하나의 트리가 가질 수 있는 최대 leaf의 개수

max_depth: 트리가 가질 수 있는 최대 depth, 0 이하로 설정하면 limit 없음

learning_rate: boosting step에서의 학습률, 0~1 사이의 값

n_estimators: fitting을 진행할 boosted tree의 개수

objective: learning task 및 목적함수 설정

In [8]:
lgbm.fit(Xdata, Ydata)

LGBMClassifier(learning_rate=0.001, n_estimators=1000, num_leaves=100,
               objective='binary')

In [9]:
pred_lgbm = lgbm.predict(Xdata)

In [10]:
# scoring은 macro f1 score를 이용
f1_score(Ydata, pred_lgbm, average='macro')

0.9847912553414335

# XGBoost

In [11]:
from xgboost import XGBClassifier

In [17]:
xgb = XGBClassifier(eta=0.001, max_depth = 10, objective='binary:logistic')

eta: boosting step에서의 learning rate, 0~1 사이의 값

max_depth: tree의 최대 depth

objective: learning task 및 목적함수 설정

(사용하진 않았지만)

scale_pos_weight: imbalanced data에 주로 사용, 일반적으로 sum(negative instances) / sum(positive instances)로 지정

gamma: 트리에서 추가적으로 가지를 나눌지를 결정할 최소 손실 감소 값

In [18]:
xgb.fit(Xdata, Ydata)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.001, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [19]:
pred_xgb = xgb.predict(Xdata)

In [20]:
# scoring은 macro f1 score를 이용
f1_score(Ydata, pred_xgb, average='macro')

0.9411170940957352

# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf = RandomForestClassifier(n_estimators = 200, criterion = 'log_loss', max_depth=7, max_features = 'log2')

n_estimators: tree의 개수

criterion: qualtiy of split을 측정하는 함수, gini/entropy/log_loss 

max_depth: tree의 최대 깊이

max_features: best split을 찾을 때 고려할 최대 feature 개수



In [29]:
rf.fit(Xdata, Ydata)

RandomForestClassifier(criterion='log_loss', max_depth=7, max_features='log2',
                       n_estimators=200)

In [30]:
pred_rf = rf.predict(Xdata)

In [31]:
# scoring은 macro f1 score를 이용
f1_score(Ydata, pred_rf, average='macro')

0.9458299545815331